In [1]:
print("hello world")

hello world


### zshrc에 key 숨기기(MAC)

```
echo "export OPENAI_API_KEY=<api_key>" >> ~/.zshrc
source ~/.zshrc
echo $OPENAI_API_KEY
```

* 윈도우는 그냥 환경변수에 추가하면 됨. 
    * 그후 터미널에서 ```echo %OPENAI_API_KEY%```  로 확인
    * powershell은 ```echo $env:OPENAI_API_KEY```


In [2]:
import os

from openai import OpenAI

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

client = OpenAI(api_key=OPENAI_API_KEY)

completion = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{
        'role': 'user',
        'content' : 'hi',
    }],
    temperature=0.0
)

print(completion.choices[0].message.content)

Hello! How can I assist you today?


### 데이터 전처리 함수 작성

리뷰 JSON 파일 로드

In [6]:
import json
# 윈도우는 인코딩 필요
# with open('./res/reviews.json', 'r') as f:
with open('./res/reviews.json', 'r', encoding='utf-8') as f:
    review_list = json.load(f)

review_list[:3]

[{'review': '위치 하나만큼은 굿\n엄마랑 아주 만족하고 썻어요', 'stars': 5, 'date': '1일 전'},
 {'review': '이틀동안 진짜 잘 쉬다가 놀다가 갑니다\n숙박도 좋았고 쉬는것도 좋았습니다 친절하게 설명해주시고 너무 감사합니다 앞으로 서울에선 여기로 가기로 정했습니다 감사합니다',
  'stars': 5,
  'date': '2024.10.10'},
 {'review': '전반적으로 만족스러웠으나\n수압이 약해도 너무 약함.... 정말 역대급...\n수압 때문에 재방문 하지 않을 것임',
  'stars': 4,
  'date': '2024.09.29'}]

좋은 평점과 나쁜 평점 구분
* 좋은 평점 : 별 5개
* 나쁜 평점 : 4개 이하

In [7]:
good_cnt, bad_cnt = 0, 0
for r in review_list : 
    if r['stars'] ==5:
        good_cnt += 1
    else:
        bad_cnt += 1

good_cnt, bad_cnt

(179, 41)

In [8]:
# 좋은 리뷰 나쁜 리뷰로 구분 후 저장
reviews_good, reviews_bad = [], []

for  r in review_list:
    if r['stars'] == 5:
        # 리뷰의 시작과 끝 명시(스페셜토큰)
        reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
    else:
        reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

reviews_bad[:3]

['[REVIEW_START]전반적으로 만족스러웠으나\n수압이 약해도 너무 약함.... 정말 역대급...\n수압 때문에 재방문 하지 않을 것임[REVIEW_END]',
 '[REVIEW_START]가격대비 뷰를 제외하고 별로였음. 넷플릭스도 특정객실 외에 안되나봄... 직원들은 친절하나 호텔보다는 돗대기 시장 같은 느낌이 남...[REVIEW_END]',
 '[REVIEW_START]객실연결 엘베불편 벽지가 울렁 다 떠있음\n그래도 참았는데\n24시간스테이하는데 중간에 문열고 허락없이 청소하고나가심  외출했다가 다시들어와서 깜놀 믿고 문잠그고 캐리어 다두고 나갔다가\n어이없음!!황당 그자체\n사과는 엄청하심!! 그래도 찜찜해서 넘 불쾌함!!![REVIEW_END]']

In [9]:
# 리스트 -> 하나로 합침
reviews_good_text = '/n'.join(reviews_good)
reviews_bad_text = '/n'.join(reviews_bad)

reviews_bad_text[:100]

'[REVIEW_START]전반적으로 만족스러웠으나\n수압이 약해도 너무 약함.... 정말 역대급...\n수압 때문에 재방문 하지 않을 것임[REVIEW_END]/n[REVIEW_STA'

In [10]:
import datetime
from dateutil import parser

# 전처리 함수 작성
def preprocess_reviews(path='./res/reviews.json'):
    with open(path, 'r', encoding='utf-8') as f:
        review_list = json.load(f)

    reviews_good, reviews_bad = [], []

    # 최근 6개월치만 가져옴
    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)

    for r in review_list:
        review_date_str = r['date']
        # 날짜로 표시가 아닌 '22시간전' '2일전' 등으로 표시된 데이터 변환
        try: 
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError):
            review_date = current_date

        if review_date < date_boundary:
            continue
        
        if r['stars'] == 5:
            # 리뷰의 시작과 끝 명시(스페셜토큰)
            reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
    
    reviews_good_text = '/n'.join(reviews_good)
    reviews_bad_text = '/n'.join(reviews_bad)

    return reviews_good_text, reviews_bad_text
        

good, bad = preprocess_reviews()
good[:100]

'[REVIEW_START]위치 하나만큼은 굿\n엄마랑 아주 만족하고 썻어요[REVIEW_END]/n[REVIEW_START]이틀동안 진짜 잘 쉬다가 놀다가 갑니다\n숙박도 좋았고 쉬는'

### 평가용 함수 작성

평가 기준 설정
* MT-Bench 논문 기반 Pairwise Comparison (=LLM 기반 평가)
    * 비교하는 방식 vs 점수 매기는 방식
    * 점수라는게 애매 할 수 있음(ex. 어느 정도의 요약 품질이 3점인가?)
    * 경험상 점수보다는 비교가 상대적으로 더 정확한 편

폄가 스크립트 작성
* MT-Bench 논문 Prompt에서 일부 단어만 수정(ex. Korean Summaries, accommodations)
    * 프롬프트가 조금이라도 깨지면 안되기 때문
    * https://arxiv.org/pdf/2306.05685.pdf (A. Prompt Templates Figure 5)
    * https://github.com/lm-sys/FastChat/blob/main/fastchat/llm_judge/data/judge_prompts.jsonl

In [11]:
def pairwise_eval(reviews, answer_a, answer_b):
    eval_prompt = f"""[System]
Please act as an impartial judge and evaluate the quality of the Korean summaries provided by two
AI assistants to the set of user reviews on accommodations displayed below. You should choose the assistant that
follows the user’s instructions and answers the user’s question better. Your evaluation
should consider factors such as the helpfulness, relevance, accuracy, depth, creativity,
and level of detail of their responses. Begin your evaluation by comparing the two
responses and provide a short explanation. Avoid any position biases and ensure that the
order in which the responses were presented does not influence your decision. Do not allow
the length of the responses to influence your evaluation. Do not favor certain names of
the assistants. Be as objective as possible. After providing your explanation, output your
final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"
if assistant B is better, and "[[C]]" for a tie.
[User Reviews]
{reviews}
[The Start of Assistant A’s Answer]
{answer_a}
[The End of Assistant A’s Answer]
[The Start of Assistant B’s Answer]
{answer_b}
[The End of Assistant B’s Answer]"""
    
    # 모델기반 평가
    completion = client.chat.completions.create(
        model='gpt-4o-2024-05-13',
        messages=[{'role': 'user', 'content': eval_prompt}],
        temperature=0.0
    )

    return completion

### Baseline 모델 개발

* 프롬프트는 영어 성능이 좋기 때문에 원래라면 그냥 영어로 한다.
* 모델 기반으로 할때에는 50~100번 이상은 해야 성능이 나온다. 

In [13]:

PROMPT_BASELINE = f"""아래 숙소 리뷰에 대해 5문장 내로 요약해줘:"""

In [15]:
reviews, _ = preprocess_reviews(path='./res/reviews.json')

def summarize(reviews, prompt, temperature=0.0, model='gpt-3.5-turbo-0125'):
    prompt = prompt + '\n\n' + reviews

    completion = client.chat.completions.create(
        model=model,
        messages=[{'role': 'user', 'content': prompt}],
        temperature=temperature
    )

    return completion

print(summarize(reviews, PROMPT_BASELINE).choices[0].message.content)

위치가 좋고 깨끗한 숙소로 엄마와 함께 만족하며 머물렀다.


In [17]:
# 실제 요약(대조군)
summary_real_20240526 = '위치가 매우 우수한 숙박시설로, 인사동과 조계사, 경복궁 등 관광지에 도보로 이동할 수 있는 편리한 위치에 있습니다. 객실은 깔끔하며 직원들의 친절한 서비스와 청결한 시설이 인상적입니다. 주변에는 맛집과 편의시설이 많아 편리하며, 교통 접근성도 좋습니다. 전체적으로 만족도가 높고 자주 방문하고 싶은 곳으로 손꼽히는 숙소로 평가됩니다.'

In [18]:
# 대조군과 비교
print(pairwise_eval(reviews, summarize(reviews, PROMPT_BASELINE).choices[0].message.content, summary_real_20240526).choices[0].message.content)

Assistant A의 요약은 매우 간결하고, 리뷰의 일부만을 반영하고 있습니다. "위치가 좋고 깨끗한 숙소로 엄마와 함께 만족하며 머물렀다"는 문장은 리뷰의 전체적인 내용을 충분히 반영하지 못하고 있으며, 세부적인 정보나 다양한 리뷰의 의견을 포함하지 않습니다.

반면, Assistant B의 요약은 리뷰의 다양한 측면을 잘 반영하고 있습니다. 위치의 우수성, 관광지 접근성, 객실의 청결함, 직원들의 친절함, 주변 편의시설, 교통 접근성 등 여러 요소를 포함하여 종합적으로 평가하고 있습니다. 이는 리뷰의 다양한 의견을 잘 반영하고 있으며, 사용자에게 더 많은 정보를 제공해줍니다.

따라서, Assistant B의 요약이 더 도움이 되고, 리뷰의 내용을 더 정확하고 깊이 있게 반영하고 있다고 판단됩니다.

[[B]]


대규모 평가 스크립트
원래는 많은 수의 다양한 데이터에 대해 평가를 하지만, 동일한 Prompt에 대해 temperature 높여서 평가 진행

In [19]:
# 10번 진행
eval_count = 10

summaries_baseline = [summarize(reviews, PROMPT_BASELINE, temperature=1.0).choices[0].message.content for _ in range(eval_count)]
summaries_baseline

['서울 안국역 인근의 숙소에서는 위치가 좋고, 깨끗하며 친절한 직원이 있어 만족스러운 숙박을 할 수 있다.',
 '위치 좋고 깨끗한 숙소, 친절한 직원과 편안한 휴식 즐길 수 있음.',
 '다양한 리뷰 중 가장 많이 언급된 내용은 위치의 좋음과 깨끗한 숙소, 친절한 직원들에 대한 언급이며, 주변 관광명소와 먹거리가 풍부한 위치로 소개되고 있습니다.',
 '다양한 리뷰들이 있지만 대부분 숙소의 위치와 깨끗함, 친절한 직원에 대해 만족하고 있다.',
 '위치가 좋고 깨끗한 숙소로 엄마와 함께 만족스러운 숙박을 즐겼다.',
 '숙소 위치가 좋고 엄마와 함께 만족했던 리뷰를 받음. 방도 깨끗하고 직원도 친절하다고 평가 받았으며, 주변에 먹을 곳과 볼거리가 풍부한 것으로 알려져 있음.',
 '위치 좋고 깨끗하니 만족스러웠던 숙박. 가격도 합리적이라 재방문 의사 있음.',
 '위치 좋고, 깨끗하고 친절한 숙소로 엄마와 만족하며 편안하게 놀다가 감니다.',
 '위치가 좋고, 숙소는 깔끔하며 직원도 친절한데 주말 가격이 조금 높다는 후기였습니다.',
 '위치 좋고 깨끗한 호텔, 친절한 직원, 잘 쉬다가 간다, 앞으로 재방문하기로 결정함, 감사합니다.']

tqdm 설치
```
pip install tqdm
```

In [20]:
from tqdm import tqdm


def pairwise_eval_batch(reviews, answers_a, answers_b):
    a_cnt, b_cnt, draw_cnt = 0, 0, 0
    for i in tqdm(range(len(answers_a))):
        completion = pairwise_eval(reviews, answers_a[i], answers_b[i])
        # 최종 결과 텍스트
        verdict_text = completion.choices[0].message.content

        if '[[A]]' in verdict_text:
            a_cnt += 1
        elif '[[B]]' in verdict_text:
            b_cnt += 1
        elif '[[C]]' in verdict_text:
            draw_cnt += 1
        else:
            print('Evaluation Error')

    return a_cnt, b_cnt, draw_cnt

wins, losses, ties = pairwise_eval_batch(reviews, summaries_baseline, [summary_real_20240526 for _ in range(len(summaries_baseline))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')
# 10번해서 10번 다 진 모습

100%|██████████| 10/10 [00:53<00:00,  5.33s/it]

Wins: 0, Losses: 10, Ties: 0


### 모델 고도화 1 - 조건들 명시

In [21]:
# Role을 주고, 미션을 주고, 결과의 조건을 줌. 예시를 주면 더 잘 알아들음 

prompt = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

eval_count = 10
# 위 프롬프트를 10번 사용한 결과를 저장
summaries = [summarize(reviews, prompt, temperature=1.0).choices[0].message.content for _ in range(eval_count)]
# 대조군과 10번 비교
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20240526 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [01:24<00:00,  8.48s/it]

Wins: 2, Losses: 8, Ties: 0


모델 고도화 2 - 입력 데이터의 품질 증가
* 프롬프트의 개선보다 데이터의 품질 증가가 더 양질의 결과를 뽑을 수 있다.
* 모델보다 더 중요한 것은 데이터다. 데이터의 품질이 좋아야 성능이 무조건 좋다.

In [36]:
import datetime
from dateutil import parser

# 고품질의 리뷰만 남김(짧은 리뷰 제거,
def preprocess_reviews(path='./res/reviews.json'):
    with open(path, 'r', encoding='utf-8') as f:
        review_list = json.load(f)

    reviews_good, reviews_bad = [], []

    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)

    filtered_cnt = 0
    for r in review_list:
        review_date_str = r['date']
        try:
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError):
            review_date = current_date

        if review_date < date_boundary:
            continue
        # 짧은 리뷰 제거
        if len(r['review']) < 30:
            filtered_cnt += 1
            # print(r['review'])
            continue

        if r['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

    # 초기에는 필요 없었으나, 2shot을 할때 샘플로 추가하려 했더니 너무 많아서 토큰 제한 걸림 
    reviews_good = reviews_good[: min(len(reviews_good), 50)]
    reviews_bad = reviews_bad[: min(len(reviews_bad), 50)]

    reviews_good_text = '\n'.join(reviews_good)
    reviews_bad_text = '\n'.join(reviews_bad)
    # print(filtered_cnt)
    # print(len(reviews_good))

    return reviews_good_text, reviews_bad_text

reviews, _ = preprocess_reviews()


In [29]:
eval_count = 10
# 위 프롬프트를 10번 사용한 결과를 저장
summaries = [summarize(reviews, prompt, temperature=1.0).choices[0].message.content for _ in range(eval_count)]
# 대조군과 10번 비교
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20240526 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [02:07<00:00, 12.72s/it]

Wins: 5, Losses: 5, Ties: 0


### 모델 고도화-3 Few-shot Prompting
* 참고 데이터를 위해 데이터가 더 필요하다.(위에는 하나의 숙소에 대한 것 뿐)
    * crawler.py 수정(하나 더 만듦)

In [30]:
# 1 shot 요약 결과를 프롬프트를 변경해서 만듦 (실제 데이터를 가져와도 됨)
reviews_1shot, _ = preprocess_reviews('./res/ninetree_pangyo.json')
summary_1shot = summarize(reviews_1shot, prompt, temperature=0.0, model='gpt-4-turbo-2024-04-09').choices[0].message.content

# 요약 예시를 추가함
# 동기호출이라 느리지만, 비동기로 하면 빠름(혹시라도 잘못 작성해서 여려번 호출 되는 것을 방지하기 위해 동기)

prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.

다음은 리뷰들과 요약 예시입니다.
예시 리뷰들:
{reviews_1shot}
예시 요약 결과:
{summary_1shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [summarize(reviews, prompt, temperature=1.0, model='gpt-3.5-turbo-0125').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20240526 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [02:42<00:00, 16.30s/it]

Wins: 5, Losses: 5, Ties: 0


In [31]:
summaries

['이틀동안 진짜 잘 쉬다가 놀다가 갑니다. 숙박도 좋았고 쉬는것도 좋았습니다. 친절하게 설명해주시고 너무 감사합니다. 앞으로 서울에선 여기로 가기로 정했습니다. 감사합니다. 객실이 깨끗하고 직원도 친절해요. 침구가 깨끗하고 아주 좋았어요. 직원분들도 너무 친절하고 방도 너무 쾌적하고 좋았습니다. 숙소도 청결했고 위치도 간편하고 주변에 먹거리나 구경할 곳도 많아서 좋아요. 위치가 너무 좋아서 그 외 자잘한 단점이 다 상쇄되네요. 주말 가격 살짝 더 낮았으면 좋겠어요. 최고에요! 잘 쉬다갑니다. 주변 볼거리도 많고 호텔도 깨끗해요.',
 '이틀동안 너무 편히 쉬고 놀다가 떠납니다. 숙박이 좋았고 친절한 안내에 감사합니다. 서울에서는 앞으로 이곳으로 갈 것으로 결정했습니다. 깨끗한 객실과 친절한 직원들이 인상적이었습니다. 숙소도 청결하고 위치도 편리해서 매우 만족스러웠습니다. 위치가 너무 좋아서 다른 단점들이 상쇄되었고, 주말 가격이 조금만 낮았으면 더 좋겠다는 의견도 있네요.',
 '서울에서의 피로를 푸는 좋은 장소로 평가받고 있으며, 직원의 친절함과 청결한 객실이 언급되어 있습니다. 지리적인 요건과 편의시설로 만족도를 높이고 있는 호텔로, 서울 방문 시에 이용할만한 곳으로 권해드립니다.',
 '고객님의 리뷰를 종합해보면, 이틀 동안 즐겁고 편안한 시간을 보내셨고 숙소와 직원에 대한 칭찬을 주셨습니다. 주변 관광지와 음식점이 풍부한 위치라고 하셨고, 앞으로 서울 방문 시 항상 이곳을 선택하겠다고 결심하셨습니다. 객실이 깨끗하고 직원이 친절하다는 점도 덧붙였습니다. 합리적인 가격과 좋은 위치, 친절한 직원들이 있는 나인트리인사동은 즐겁고 편안한 숙소라고 리뷰해주셨습니다.',
 '고객님의 리뷰를 종합하면, 이틀 동안 편안하게 쉬고 놀다 오셨습니다. 숙소는 깨끗하고 침구도 편안했으며 직원들도 친절하게 응대해주셨습니다. 위치가 좋아 주변 관광지와 먹거리가 풍부한 곳이라 만족하실 것으로 예상됩니다.당신의 다음 서울 숙소로 기억하셨으면 좋겠습니다. 감사합니다.',
 '이틀

꼭 둘다(조건 선정 및 예시) 지정한다고 해서 좋아지는 것이 아님. 상황에 따라 하나만 하는게 나을 수도 있음 

In [32]:
# 조건 뺌

prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 다음은 리뷰들과 요약 예시입니다.
예시 리뷰들:
{reviews_1shot}
예시 요약 결과:
{summary_1shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [summarize(reviews, prompt_1shot, temperature=1.0, model='gpt-3.5-turbo-0125').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20240526 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [02:08<00:00, 12.81s/it]

Wins: 7, Losses: 3, Ties: 0


In [33]:
summaries

['이 숙소는 청결하고 편리한 위치를 자랑하며, 직원들의 친절한 서비스와 객실의 깨끗함에 대해 많은 만족을 얻고 있습니다. 객실의 쾌적함과 주변 관광지와의 접근성이 좋다는 점이 강조되었습니다. 또한, 주변에 많은 먹거리와 즐길 거리가 있어 편안한 여행을 즐길 수 있다고 평가받고 있습니다. 최상의 위치와 서비스를 제공하는 숙소로 여행객들에게 매우 인기가 높은 것으로 보입니다.',
 '이 숙소는 정말 잘 쉬다가 놀기에 좋은 곳으로, 청결한 객실과 친절한 직원이 인상적이었습니다. 위치가 편리하고 주변에 먹을거리와 구경할 곳이 풍부하여 만족한 이용객들이 많았습니다. 다만 조금 불편한 점은 방음이 부족했다는 의견이 있었습니다. 이 숙소는 가성비 좋고 재방문할 의사가 높은 곳으로 추천됩니다.',
 '이 숙소는 청결하고 위치가 편리하여 여러 관광지를 즐기기 좋은 장점을 가지고 있습니다. 고객들은 객실의 청결함과 직원들의 친절함에 대해 언급하며 만족을 표현하고 있습니다. 또한, 주변에 많은 먹거리와 구경할만한 장소들이 있어 즐거운 여행을 보낼 수 있는 장점이 있습니다. 가격대비 만족도가 높은 숙소로 추천합니다.',
 '이 숙소는 청결한 객실과 친절한 직원들로 인해 많은 이용객들이 만족하며, 주변 관광지와 교통편이 편리한 위치에 있어요. 또한, 고객센터에서의 친절한 응대와 좋은 시설로 인해 계속해서 재방문하는 고객들도 많습니다. 주변에 먹을거리와 구경할 곳이 많아 가족 또는 커플 여행에 적합합니다. 다만, 방음이 조금 부족하다는 의견이 있으니 참고하실 필요가 있겠습니다.',
 '이 숙소는 청결한 객실과 친절한 직원들로 인정받고 있습니다. 위치가 편리하고 주변에 먹거리와 볼거리가 풍부하여 이용객들이 만족하고 있습니다. 다만 일부 이용객은 방음에 조금 불만을 표시하였습니다. 종합적으로는 편리하고 저렴한 숙소로 평가받고 있으며, 특히 서울 갈 때마다 이용을 고민하는 많은 이용객들에게 추천되는 숙소입니다.',
 '이 숙소는 청결하고 위치가 편리하며 주변에 맛집과 관광지가 다양하게 있

In [37]:
# 2shot에 그냥 1shot 넣으려고 했더니, 토큰제한 갯수에 걸려서, 위에 코드 수정하고 새로 돌리는 셀
reviews_1shot, _ = preprocess_reviews('./res/ninetree_pangyo.json')
summary_1shot = summarize(reviews_1shot, prompt, temperature=0.0, model='gpt-4-turbo-2024-04-09').choices[0].message.content


In [38]:
reviews_2shot, _ = preprocess_reviews('./res/ninetree_yongsan.json')
# 1shot으로 더 좋은 결과를 얻기 위해 대조군이 1shot
summary_2shot = summarize(reviews_2shot, prompt_1shot, temperature=0.0, model='gpt-4-turbo-2024-04-09').choices[0].message.content

prompt_2shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 다음은 리뷰들과 요약 예시입니다.

예시 리뷰들 1:
{reviews_1shot}
예시 요약 결과 1:
{summary_1shot}

예시 리뷰들 2:
{reviews_2shot}
예시 요약 결과 2:
{summary_2shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [summarize(reviews, prompt_2shot, temperature=1.0, model='gpt-3.5-turbo-0125').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20240526 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [01:36<00:00,  9.69s/it]

Wins: 8, Losses: 2, Ties: 0


In [39]:
summaries

['이 숙소는 전반적으로 매우 만족스러웠다는 평가를 받고 있습니다. 이틀간의 투숙 기간 동안 편안히 쉬고 놀기에 좋은 휴식처로서 침구와 객실의 청결도를 긍정적으로 언급하고 있습니다. 또한, 종로, 광화문, 경복궁, 광장시장, 인사동 등의 관광 명소에 가깝고, 교통 접근성이 우수하다고 평가하고 있습니다. 주변에 편의 시설과 먹거리가 풍부하며, 프론트 직원의 친절함과 서비스도 탄탄하게 언급되었습니다. 다만, 먼지나 머리카락 등에 대한 경험적인 언급이 있었으나, 대체로 다양하고 긍정적인 피드백들로 구성되어 있습니다. 이 숙소는 추천 받을 만한 가치가 있는 곳으로 평가됩니다.',
 '이 숙소는 위치가 우수하고 깨끗하며 친절한 직원들이 인상적입니다. 객실은 깔끔하고 침구도 편안하며, 주변에 먹거리와 관광지가 풍부하여 편리한 숙박을 즐길 수 있습니다. 주차장 이용이 편리하며, 청결도와 서비스 모두 만족스러운 편이며, 대부분의 이용객들이 만족하고 재방문 의사가 높은 것으로 보입니다. 추가로 욕조가 필요한 경우 미리 요청하는 것이 좋습니다.',
 '요약 결과:\n이 숙소는 편리한 위치와 친절한 직원, 깨끗한 객실 등으로 고객들로부터 만족스러운 평가를 받고 있습니다. 근처의 명소와 교통편이 좋아 여행객들에게 좋은 선택지가 될 것으로 보입니다. 객실 청결과 내부 시설에 대한 긍정적인 평가가 많이 있으며, 일부 고객은 주변 지역의 이동이 편리하다고 언급하며 호텔 위치의 장점을 강조했습니다. 이 숙소는 인사동을 방문하는 여행객들에게 특히 좋은 선택이 될 것으로 보입니다.',
 '요청하신 리뷰들을 종합하여 아래와 같은 요약 결과를 제공해 드립니다:\n\n이 숙소는 서울 여행을 즐기기에 최적의 위치와 교통편을 제공합니다. 직원들은 친절하며, 객실은 깨끗하고 침구류가 편안합니다. 조식은 맛있고 다양한 메뉴들이 준비되어 있습니다. 숙소 주변에는 다양한 관광지와 맛집이 위치해 있어 만족스러운 여행을 즐길 수 있습니다. 위치, 청결도, 서비스, 그리고 가격대까지 모든 면에서 이용객들에게 높은